# Deep cortical reinforcement learning: Policy gradients + Q-learning + GAN


In this notebook, we'll build a neural network that can learn to play games through reinforcement learning. More specifically, we'll use Q-learning to train an agent to play a game called [Cart-Pole](https://gym.openai.com/envs/CartPole-v0). In this game, a freely swinging pole is attached to a cart. The cart can move to the left and right, and the goal is to keep the pole upright as long as possible.

![Cart-Pole](assets/cart-pole.jpg)

We can simulate this game using [OpenAI Gym](https://gym.openai.com/). First, let's check out how OpenAI Gym works. Then, we'll get into training an agent to play the Cart-Pole game.

In [63]:
# In this one we should define and detect GPUs for tensorflow
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


>**Note:** Make sure you have OpenAI Gym cloned into the same directory with this notebook. I've included `gym` as a submodule, so you can run `git submodule --init --recursive` to pull the contents into the `gym` repo.

##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [64]:
import gym

## Create the Cart-Pole game environment
# env = gym.make('CartPole-v0')
# env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('MountainCarContinuous-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

# Discrete/int or continuos/float
env.action_space.dtype

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


dtype('float32')

We interact with the simulation through `env`. To show the simulation running, you can use `env.render()` to render one frame. Passing in an action as an integer to `env.step` will generate the next step in the simulation.  You can see how many actions are possible from `env.action_space` and to get a random action you can use `env.action_space.sample()`. This is general to all Gym games. In the Cart-Pole game, there are two possible actions, moving the cart left or right. So there are two actions we can take, encoded as 0 and 1.

Run the code below to watch the simulation run.

In [65]:
env.observation_space

Box(24,)

In [66]:
state = env.reset()
batch = []
for _ in range(1111):
    #env.render()
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action) # take a random action
    batch.append([state, action, next_state, reward, float(done)])
    state = next_state
    if done:
        state = env.reset()

To shut the window showing the simulation, use `env.close()`.

In [67]:
# env.close()

If you ran the simulation above, we can look at the rewards:

In [68]:
batch[0], 
batch[0][1].shape, state.shape

((4,), (24,))

In [69]:
import numpy as np
actions = np.array([each[0] for each in batch])
states = np.array([each[1] for each in batch])
rewards = np.array([each[2] for each in batch])
dones = np.array([each[3] for each in batch])
infos = np.array([each[4] for each in batch])

In [70]:
# print(rewards[-20:])
print('shapes:', np.array(rewards).shape, np.array(states).shape, np.array(actions).shape, np.array(dones).shape)
print('dtypes:', np.array(rewards).dtype, np.array(states).dtype, np.array(actions).dtype, np.array(dones).dtype)
print('states:', np.max(np.array(states)), np.min(np.array(states)))
print('actions:', np.max(np.array(actions)), np.min(np.array(actions)))
# print((np.max(np.array(actions)) - np.min(np.array(actions)))+1)
print('rewards:', np.max(np.array(rewards)), np.min(np.array(rewards)))

shapes: (1111, 24) (1111, 4) (1111, 24) (1111,)
dtypes: float64 float32 float64 float64
states: 0.9999491 -0.99980396
actions: 2.0423901875813804 -1.386185646057129
rewards: 2.0423901875813804 -1.386185646057129


In [71]:
actions[:10]

array([[ 2.74739973e-03, -1.31684646e-05,  1.02436312e-03,
        -1.59999108e-02,  9.19683278e-02, -1.35180948e-03,
         8.60266164e-01,  2.40187673e-03,  1.00000000e+00,
         3.23748849e-02, -1.35171739e-03,  8.53814155e-01,
         9.56013566e-04,  1.00000000e+00,  4.40814108e-01,
         4.45820212e-01,  4.61422890e-01,  4.89550292e-01,
         5.34102917e-01,  6.02461159e-01,  7.09149063e-01,
         8.85932028e-01,  1.00000000e+00,  1.00000000e+00],
       [ 1.81099847e-02,  2.27325392e-02,  1.50036299e-02,
         1.87298584e-02, -3.19512904e-01, -9.25155520e-01,
         1.48349273e+00,  9.93954817e-01,  1.00000000e+00,
         2.91940302e-01,  1.52260810e-01,  1.36669636e-01,
        -9.99908447e-01,  1.00000000e+00,  4.51477766e-01,
         4.56604987e-01,  4.72585082e-01,  5.01392901e-01,
         5.47023296e-01,  6.17035210e-01,  7.26303935e-01,
         9.07363415e-01,  1.00000000e+00,  1.00000000e+00],
       [ 3.07344068e-02,  2.62205172e-02,  1.24771661e

In [72]:
rewards[:10]

array([[ 1.81099847e-02,  2.27325392e-02,  1.50036299e-02,
         1.87298584e-02, -3.19512904e-01, -9.25155520e-01,
         1.48349273e+00,  9.93954817e-01,  1.00000000e+00,
         2.91940302e-01,  1.52260810e-01,  1.36669636e-01,
        -9.99908447e-01,  1.00000000e+00,  4.51477766e-01,
         4.56604987e-01,  4.72585082e-01,  5.01392901e-01,
         5.47023296e-01,  6.17035210e-01,  7.26303935e-01,
         9.07363415e-01,  1.00000000e+00,  1.00000000e+00],
       [ 3.07344068e-02,  2.62205172e-02,  1.24771661e-02,
        -8.54406476e-03, -8.12581480e-02, -8.26262236e-01,
         1.10302541e+00,  0.00000000e+00,  1.00000000e+00,
         3.13992411e-01,  3.77926439e-01,  4.68376875e-02,
        -1.00000008e+00,  0.00000000e+00,  4.53954518e-01,
         4.59109843e-01,  4.75177616e-01,  5.04143476e-01,
         5.50024211e-01,  6.20420158e-01,  7.30288327e-01,
         9.12341118e-01,  1.00000000e+00,  1.00000000e+00],
       [ 3.42181735e-02,  6.00428879e-03,  1.74115118e

In [73]:
# import numpy as np
def sigmoid(x, derivative=False):
  return x*(1-x) if derivative else 1/(1+np.exp(-x))

In [74]:
sigmoid(np.max(np.array(rewards))), sigmoid(np.min(np.array(rewards)))

(0.8851764282630248, 0.20001739497735352)

In [75]:
print('rewards:', np.max(np.array(rewards))/100, np.min(np.array(rewards))/100)

rewards: 0.020423901875813805 -0.013861856460571288


The game resets after the pole has fallen past a certain angle. For each frame while the simulation is running, it returns a reward of 1.0. The longer the game runs, the more reward we get. Then, our network's goal is to maximize the reward by keeping the pole vertical. It will do this by moving the cart to the left and the right.

## Q-Network

We train our Q-learning agent using the Bellman Equation:

$$
Q(s, a) = r + \gamma \max{Q(s', a')}
$$

where $s$ is a state, $a$ is an action, and $s'$ is the next state from state $s$ and action $a$.

Before we used this equation to learn values for a Q-_table_. However, for this game there are a huge number of states available. The state has four values: the position and velocity of the cart, and the position and velocity of the pole. These are all real-valued numbers, so ignoring floating point precisions, you practically have infinite states. Instead of using a table then, we'll replace it with a neural network that will approximate the Q-table lookup function.

<img src="assets/deep-q-learning.png" width=450px>

Now, our Q value, $Q(s, a)$ is calculated by passing in a state to the network. The output will be Q-values for each available action, with fully connected hidden layers.

<img src="assets/q-network.png" width=550px>


As I showed before, we can define our targets for training as $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$. Then we update the weights by minimizing $(\hat{Q}(s,a) - Q(s,a))^2$. 

For this Cart-Pole game, we have four inputs, one for each value in the state, and two outputs, one for each action. To get $\hat{Q}$, we'll first choose an action, then simulate the game using that action. This will get us the next state, $s'$, and the reward. With that, we can calculate $\hat{Q}$ then pass it back into the $Q$ network to run the optimizer and update the weights.

Below is my implementation of the Q-network. I used two fully connected layers with ReLU activations. Two seems to be good enough, three might be better. Feel free to try it out.

In [76]:
def model_input(state_size, action_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.float32, [None, action_size], name='actions')
    targetQs = tf.placeholder(tf.float32, [None], name='targetQs')
    return states, actions, targetQs

In [77]:
# How to use batch-norm
#   x_norm = tf.layers.batch_normalization(x, training=training)

#   # ...

#   update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
#   with tf.control_dependencies(update_ops):
#     train_op = optimizer.minimize(loss)

In [78]:
# training: Either a Python boolean, or a TensorFlow boolean scalar tensor (e.g. a placeholder). 
# Whether to return the output in: 
# training mode (normalized with statistics of the current batch) or 
# inference mode (normalized with moving statistics). 
# NOTE: make sure to set this parameter correctly, or else your training/inference will not work properly.

In [79]:
# Generator/Controller: Generating/prediting the actions
def generator(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        #predictions = tf.nn.softmax(logits)

        # return actions logits
        return logits

In [80]:
# Discriminator/Dopamine: Reward function/planner/naviator/advisor/supervisor/cortical columns
def discriminator(states, actions, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # Fusion/merge states and actions/ SA/ SM
        x_fused = tf.concat(axis=1, values=[states, actions])
        
        # First fully connected layer
        h1 = tf.layers.dense(inputs=x_fused, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        #predictions = tf.nn.softmax(logits)

        # return rewards logits
        return logits

In [81]:
batch[1][1], batch[2][1], np.square(batch[1][1] - batch[2][1])

(array([-0.6857498 , -0.1863178 ,  0.5463643 , -0.48135957], dtype=float32),
 array([ 0.76391673,  0.2900372 , -0.20683765, -0.2392354 ], dtype=float32),
 array([2.101533  , 0.22691411, 0.5673132 , 0.05862411], dtype=float32))

In [82]:
def model_loss(action_size, hidden_size, states, actions, targetQs):
    # G
    actions_logits = generator(states=states, hidden_size=hidden_size, action_size=action_size)
    #actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    actions_labels = tf.nn.sigmoid(actions)
    # neg_log_prob_actions = tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
    #                                                                   labels=actions_labels)
    neg_log_prob_actions = tf.nn.sigmoid_cross_entropy_with_logits(logits=actions_logits, 
                                                                   labels=actions_labels)
    #g_loss = tf.reduce_mean(neg_log_prob_actions * targetQs) # error!
    
    # D
    Qs = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states)
    d_loss = tf.reduce_mean(tf.square(Qs - targetQs))
    g_loss = tf.reduce_mean(neg_log_prob_actions * Qs)
    return actions_logits, Qs, g_loss, d_loss

In [90]:
# Optimizating/training/learning G & D
def model_opt(g_loss, d_loss, learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        g_opt = tf.train.AdamOptimizer(learning_rate).minimize(g_loss, var_list=g_vars)
        d_opt = tf.train.AdamOptimizer(learning_rate).minimize(d_loss, var_list=d_vars)

    return g_opt, d_opt

In [91]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.targetQs = model_input(state_size=state_size, action_size=action_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.Qs_logits, self.g_loss, self.d_loss = model_loss(
            action_size=action_size, hidden_size=hidden_size, # model init parameters
            states=self.states, actions=self.actions, targetQs=self.targetQs) # model input
        
        # Update the model: backward pass and backprop
        self.g_opt, self.d_opt = model_opt(g_loss=self.g_loss, d_loss=self.d_loss, learning_rate=learning_rate)

In [92]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size)

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [93]:
print('state size:{}'.format(states.shape), 
      'actions:{}'.format(actions.shape)) 
print('action size:{}'.format(np.max(actions) - np.min(actions)+1))

state size:(1111, 4) actions:(1111, 24)
action size:4.42857583363851


In [94]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01            # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
# state_size = 37
# state_size_ = (84, 84, 3)
state_size = 24
action_size = 4
hidden_size = 24*2             # number of units in each Q-network hidden layer
learning_rate = 0.0001         # Q-network learning rate

# Memory parameters
gamma = 0.99                   # future reward discount
memory_size = 1000            # memory capacity
batch_size = 1000             # experience mini-batch size

In [95]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, learning_rate=learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [96]:
state = env.reset()
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    memory.buffer.append([state, action, next_state, reward, float(done)])
    state = next_state
    if done is True:
        state = env.reset()

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list, gloss_list, dloss_list = [], [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window
    
    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0
        gloss_batch, dloss_batch = [], []
        state = env.reset()

        # Training steps/batches
        while True:
            # Explore (Env) or Exploit (Model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.reshape(action_logits, [-1]) # For continuous action space
                #action = np.argmax(action_logits) # For discrete action space
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done)])
            total_reward += reward
            state = next_state

            # Training
            #batch = memory.sample(batch_size)
            batch = memory.buffer
            states = np.array([each[0] for each in batch])
            actions = np.array([each[1] for each in batch])
            next_states = np.array([each[2] for each in batch])
            rewards = np.array([each[3] for each in batch])
            dones = np.array([each[4] for each in batch])
            nextQs_logits = sess.run(model.Qs_logits, feed_dict = {model.states: next_states})
            #nextQs = np.max(nextQs_logits, axis=1) * (1-dones)
            nextQs = nextQs_logits.reshape([-1]) * (1-dones)
            targetQs = rewards + (gamma * nextQs)
            gloss, dloss, _, _ = sess.run([model.g_loss, model.d_loss, model.g_opt, model.d_opt],
                                            feed_dict = {model.states: states, 
                                                         model.actions: actions,
                                                         model.targetQs: targetQs})
            gloss_batch.append(gloss)
            dloss_batch.append(dloss)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'gloss:{:.4f}'.format(np.mean(gloss_batch)),
              'dloss:{:.4f}'.format(np.mean(dloss_batch)),
              'exploreP:{:.4f}'.format(explore_p))
        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        gloss_list.append([ep, np.mean(gloss_batch)])
        dloss_list.append([ep, np.mean(dloss_batch)])
        # Break episode/epoch loop
        if np.mean(episode_reward) <= -150:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model2.ckpt')

Episode:0 meanR:-82.3067 R:-82.3067 gloss:-134.2165 dloss:1.8306 exploreP:0.8536
Episode:1 meanR:-91.3119 R:-100.3171 gloss:-388.0928 dloss:0.2071 exploreP:0.8484
Episode:2 meanR:-100.4618 R:-118.7616 gloss:-753.1843 dloss:9.0228 exploreP:0.8412
Episode:3 meanR:-100.9111 R:-102.2588 gloss:-2239.4990 dloss:8.6724 exploreP:0.7183
Episode:4 meanR:-106.8413 R:-130.5623 gloss:-1098.4009 dloss:0.0550 exploreP:0.6136
Episode:5 meanR:-107.8460 R:-112.8692 gloss:-1225.1321 dloss:0.1985 exploreP:0.6091
Episode:6 meanR:-108.6487 R:-113.4653 gloss:-2143.2043 dloss:8.4064 exploreP:0.6054
Episode:7 meanR:-109.5934 R:-116.2061 gloss:-3168.9819 dloss:14.7852 exploreP:0.6002
Episode:8 meanR:-109.6470 R:-110.0754 gloss:-4279.8823 dloss:19.6476 exploreP:0.5953
Episode:9 meanR:-109.9836 R:-113.0138 gloss:-5097.3779 dloss:23.9260 exploreP:0.5917
Episode:10 meanR:-109.9658 R:-109.7870 gloss:-5920.8799 dloss:27.3604 exploreP:0.5894
Episode:11 meanR:-110.5111 R:-116.5096 gloss:-6912.3984 dloss:29.5833 explore

Episode:95 meanR:-117.6296 R:-118.9551 gloss:-33751.5273 dloss:19.4492 exploreP:0.1591
Episode:96 meanR:-117.6009 R:-114.8476 gloss:-33846.7148 dloss:18.8796 exploreP:0.1573
Episode:97 meanR:-117.5728 R:-114.8409 gloss:-32216.5254 dloss:19.1349 exploreP:0.1559
Episode:98 meanR:-117.5868 R:-118.9664 gloss:-30184.0801 dloss:18.5370 exploreP:0.1535
Episode:99 meanR:-117.5937 R:-118.2697 gloss:-29020.0664 dloss:18.4548 exploreP:0.1514
Episode:100 meanR:-117.9392 R:-116.8619 gloss:-27615.7617 dloss:18.9935 exploreP:0.1490
Episode:101 meanR:-118.1338 R:-119.7713 gloss:-24376.9941 dloss:18.3554 exploreP:0.1470
Episode:102 meanR:-118.0810 R:-113.4856 gloss:-22511.0645 dloss:18.6199 exploreP:0.1461
Episode:103 meanR:-118.2127 R:-115.4286 gloss:-23178.4844 dloss:19.1605 exploreP:0.1449
Episode:104 meanR:-118.0761 R:-116.9018 gloss:-22717.4043 dloss:18.8696 exploreP:0.1431
Episode:105 meanR:-118.1156 R:-116.8202 gloss:-23596.7637 dloss:18.7408 exploreP:0.1411
Episode:106 meanR:-118.2102 R:-122.92

Episode:189 meanR:-134.7846 R:-181.0995 gloss:-274.9923 dloss:0.0759 exploreP:0.0115
Episode:190 meanR:-135.3347 R:-171.9956 gloss:-1089.4066 dloss:0.3037 exploreP:0.0115
Episode:191 meanR:-135.4302 R:-130.7280 gloss:-5541.2095 dloss:8.1978 exploreP:0.0114
Episode:192 meanR:-135.4824 R:-119.1651 gloss:-6424.5894 dloss:16.1300 exploreP:0.0114
Episode:193 meanR:-135.5225 R:-129.8688 gloss:-17639.8809 dloss:22.8825 exploreP:0.0114
Episode:194 meanR:-135.6330 R:-129.9176 gloss:-42471.4414 dloss:26.8103 exploreP:0.0114
Episode:195 meanR:-135.6413 R:-119.7900 gloss:-60710.2500 dloss:33.3036 exploreP:0.0114
Episode:196 meanR:-135.6923 R:-119.9498 gloss:-58488.7148 dloss:29.9767 exploreP:0.0114
Episode:197 meanR:-135.7440 R:-120.0108 gloss:-54161.4219 dloss:33.4855 exploreP:0.0114
Episode:198 meanR:-135.7545 R:-120.0103 gloss:-55681.3789 dloss:35.8326 exploreP:0.0114
Episode:199 meanR:-135.7714 R:-119.9577 gloss:-57414.9219 dloss:32.4272 exploreP:0.0114
Episode:200 meanR:-135.8021 R:-119.9403 

Episode:284 meanR:-128.9313 R:-123.5110 gloss:-4116.3955 dloss:7.3530 exploreP:0.0107
Episode:285 meanR:-128.3659 R:-123.4096 gloss:-4614.1499 dloss:5.0619 exploreP:0.0107
Episode:286 meanR:-127.7930 R:-123.5606 gloss:-4726.7275 dloss:7.2756 exploreP:0.0107
Episode:287 meanR:-127.2231 R:-123.5904 gloss:-4993.6323 dloss:5.8186 exploreP:0.0107
Episode:288 meanR:-126.6591 R:-124.2789 gloss:-4771.0806 dloss:6.7263 exploreP:0.0107
Episode:289 meanR:-126.0837 R:-123.5595 gloss:-4821.4277 dloss:5.0775 exploreP:0.0107
Episode:290 meanR:-125.5994 R:-123.5582 gloss:-4722.2983 dloss:5.0578 exploreP:0.0107
Episode:291 meanR:-125.5266 R:-123.4568 gloss:-5101.2012 dloss:6.6891 exploreP:0.0107
Episode:292 meanR:-125.5707 R:-123.5694 gloss:-5311.0103 dloss:5.4038 exploreP:0.0107
Episode:293 meanR:-125.5074 R:-123.5440 gloss:-5316.4429 dloss:7.1010 exploreP:0.0107
Episode:294 meanR:-125.4450 R:-123.6750 gloss:-5106.9434 dloss:5.3758 exploreP:0.0107
Episode:295 meanR:-125.4833 R:-123.6199 gloss:-4971.72

Episode:379 meanR:-123.4578 R:-123.5285 gloss:-4149.0620 dloss:7.8656 exploreP:0.0105
Episode:380 meanR:-123.4540 R:-123.5626 gloss:-4164.6509 dloss:6.7130 exploreP:0.0105
Episode:381 meanR:-123.4508 R:-123.5891 gloss:-4174.3540 dloss:6.9395 exploreP:0.0105
Episode:382 meanR:-123.4469 R:-123.5667 gloss:-4218.5254 dloss:8.0204 exploreP:0.0105
Episode:383 meanR:-123.4475 R:-123.5610 gloss:-4486.2627 dloss:7.4002 exploreP:0.0105
Episode:384 meanR:-123.4477 R:-123.5370 gloss:-4159.6094 dloss:7.0571 exploreP:0.0105
Episode:385 meanR:-123.4488 R:-123.5133 gloss:-3589.9290 dloss:6.8889 exploreP:0.0105
Episode:386 meanR:-123.4338 R:-122.0626 gloss:-4105.4668 dloss:7.2557 exploreP:0.0105
Episode:387 meanR:-123.4329 R:-123.5041 gloss:-4085.1243 dloss:18.9537 exploreP:0.0105
Episode:388 meanR:-123.3207 R:-113.0544 gloss:-4720.0303 dloss:27.7959 exploreP:0.0104
Episode:389 meanR:-123.3203 R:-123.5256 gloss:-4476.3755 dloss:32.5143 exploreP:0.0104
Episode:390 meanR:-123.3200 R:-123.5262 gloss:-4006

...

Episode:3259 meanR:-119.1620 R:-123.5733 gloss:-81552.4609 dloss:47.0144 exploreP:0.0100
Episode:3260 meanR:-119.2654 R:-123.6249 gloss:-81465.9141 dloss:43.0392 exploreP:0.0100
Episode:3261 meanR:-119.3077 R:-117.5517 gloss:-85605.7109 dloss:52.3524 exploreP:0.0100
Episode:3262 meanR:-119.4085 R:-123.5208 gloss:-88669.6562 dloss:84.9983 exploreP:0.0100
Episode:3263 meanR:-119.4869 R:-123.4902 gloss:-84812.5781 dloss:43.9344 exploreP:0.0100
Episode:3264 meanR:-119.5894 R:-123.5807 gloss:-77524.4141 dloss:37.2918 exploreP:0.0100
Episode:3265 meanR:-119.6893 R:-123.5523 gloss:-76531.7969 dloss:53.7259 exploreP:0.0100
Episode:3266 meanR:-119.7898 R:-123.4989 gloss:-66114.8438 dloss:30.7689 exploreP:0.0100
Episode:3267 meanR:-119.8913 R:-123.5863 gloss:-65027.0195 dloss:54.7932 exploreP:0.0100
Episode:3268 meanR:-119.9914 R:-123.5694 gloss:-62288.0781 dloss:30.1903 exploreP:0.0100
Episode:3269 meanR:-120.0931 R:-123.4943 gloss:-55727.8203 dloss:44.7000 exploreP:0.0100
Episode:3270 meanR:-1

Episode:3351 meanR:-120.0591 R:-102.5094 gloss:-214649.9531 dloss:39.2848 exploreP:0.0100
Episode:3352 meanR:-119.8467 R:-102.3851 gloss:-172290.1250 dloss:38.7039 exploreP:0.0100
Episode:3353 meanR:-119.6398 R:-102.7642 gloss:-127342.3828 dloss:38.8527 exploreP:0.0100
Episode:3354 meanR:-119.4356 R:-103.7092 gloss:-87539.2891 dloss:38.3678 exploreP:0.0100
Episode:3355 meanR:-119.2257 R:-102.5137 gloss:-54201.3789 dloss:37.6587 exploreP:0.0100
Episode:3356 meanR:-119.0157 R:-102.4773 gloss:-50805.4336 dloss:37.2316 exploreP:0.0100
Episode:3357 meanR:-118.8113 R:-103.2354 gloss:-52546.6055 dloss:36.9757 exploreP:0.0100
Episode:3358 meanR:-118.6097 R:-103.3540 gloss:-52819.6289 dloss:36.9546 exploreP:0.0100
Episode:3359 meanR:-118.3995 R:-102.5513 gloss:-53842.0938 dloss:37.6687 exploreP:0.0100
Episode:3360 meanR:-118.1882 R:-102.5022 gloss:-52410.4844 dloss:66.8038 exploreP:0.0100
Episode:3361 meanR:-118.0390 R:-102.6226 gloss:-57243.0547 dloss:49.5419 exploreP:0.0100
Episode:3362 meanR

Episode:3443 meanR:-102.8626 R:-102.5373 gloss:-25888.9473 dloss:123.9569 exploreP:0.0100
Episode:3444 meanR:-102.8625 R:-102.5364 gloss:-19756.5801 dloss:25.9834 exploreP:0.0100
Episode:3445 meanR:-102.8627 R:-102.5215 gloss:-16432.5039 dloss:24.8156 exploreP:0.0100
Episode:3446 meanR:-102.8617 R:-102.3868 gloss:-20017.5977 dloss:48.5066 exploreP:0.0100
Episode:3447 meanR:-102.8546 R:-102.4968 gloss:-21930.6895 dloss:257.1576 exploreP:0.0100
Episode:3448 meanR:-102.8500 R:-102.5407 gloss:-22153.6133 dloss:25.7098 exploreP:0.0100
Episode:3449 meanR:-102.8505 R:-102.5522 gloss:-22131.4570 dloss:24.2241 exploreP:0.0100
Episode:3450 meanR:-102.8514 R:-102.5209 gloss:-22164.8555 dloss:24.1257 exploreP:0.0100
Episode:3451 meanR:-102.8512 R:-102.4962 gloss:-20775.3633 dloss:23.6538 exploreP:0.0100
Episode:3452 meanR:-102.8621 R:-103.4748 gloss:-22222.3105 dloss:25.7734 exploreP:0.0100
Episode:3453 meanR:-102.8595 R:-102.4973 gloss:-22889.1875 dloss:50.3112 exploreP:0.0100
Episode:3454 meanR:

Episode:3535 meanR:-102.7221 R:-102.9900 gloss:-30296.0488 dloss:35.0489 exploreP:0.0100
Episode:3536 meanR:-102.7223 R:-102.5523 gloss:-29463.2246 dloss:33.5103 exploreP:0.0100
Episode:3537 meanR:-102.7218 R:-102.5393 gloss:-29185.0469 dloss:106.4362 exploreP:0.0100
Episode:3538 meanR:-102.7214 R:-103.4004 gloss:-33094.9727 dloss:139.7803 exploreP:0.0100
Episode:3539 meanR:-102.7317 R:-103.5885 gloss:-31327.7949 dloss:39.6095 exploreP:0.0100
Episode:3540 meanR:-102.7457 R:-104.7476 gloss:-88802.6250 dloss:52.4094 exploreP:0.0100
Episode:3541 meanR:-102.7488 R:-102.8440 gloss:-160275.7969 dloss:46.0270 exploreP:0.0100
Episode:3542 meanR:-102.7909 R:-106.7590 gloss:-165856.4062 dloss:40.6591 exploreP:0.0100
Episode:3543 meanR:-102.7973 R:-103.1760 gloss:-170235.2656 dloss:38.5718 exploreP:0.0100
Episode:3544 meanR:-102.9248 R:-115.2864 gloss:-272963.8750 dloss:97.5480 exploreP:0.0100
Episode:3545 meanR:-103.6946 R:-179.4982 gloss:-144951.0000 dloss:26.0590 exploreP:0.0100
Episode:3546 m

Episode:3628 meanR:-131.5544 R:-103.4335 gloss:-83117.5547 dloss:35.2061 exploreP:0.0100
Episode:3629 meanR:-131.5990 R:-107.0139 gloss:-73801.1562 dloss:37.0594 exploreP:0.0100
Episode:3630 meanR:-131.6179 R:-104.1790 gloss:-66209.2812 dloss:38.9387 exploreP:0.0100
Episode:3631 meanR:-131.6420 R:-104.8990 gloss:-61675.5156 dloss:39.9640 exploreP:0.0100
Episode:3632 meanR:-131.6337 R:-102.5657 gloss:-48121.2383 dloss:41.3438 exploreP:0.0100
Episode:3633 meanR:-131.6424 R:-103.3787 gloss:-38915.2969 dloss:43.0994 exploreP:0.0100
Episode:3634 meanR:-131.6410 R:-103.2956 gloss:-36062.9336 dloss:44.7429 exploreP:0.0100
Episode:3635 meanR:-131.6295 R:-101.8364 gloss:-36500.2930 dloss:45.6668 exploreP:0.0100
Episode:3636 meanR:-131.6538 R:-104.9822 gloss:-36873.8672 dloss:45.6116 exploreP:0.0100
Episode:3637 meanR:-131.6532 R:-102.4766 gloss:-39065.8672 dloss:38.6661 exploreP:0.0100
Episode:3638 meanR:-131.6471 R:-102.7959 gloss:-41588.3164 dloss:35.5672 exploreP:0.0100
Episode:3639 meanR:-1

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(gloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('G losses')

In [ ]:
eps, arr = np.array(dloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [36]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(1):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.